<a href="https://colab.research.google.com/github/jasonwillschiu/scratch/blob/master/meetup%20bot%20--%20gcs%20no%20tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install gcsfs

     |████████████████████████████████| 92kB 3.0MB/s 
     |████████████████████████████████| 1.2MB 9.1MB/s 
     |████████████████████████████████| 153kB 16.9MB/s 
     |████████████████████████████████| 266kB 17.0MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3161 sha256=99963dc1bb4534c4671f1dbaca11d26f84c252b4d8ed5e22ab42575babf54410
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [8]:
import os
os.listdir()

['.config',
 'adc.json',
 'robotic-tiger-270605-60df8bc892bb.json',
 'sample_data']

In [10]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "robotic-tiger-270605-60df8bc892bb.json"

In [16]:
from google.cloud import storage
client = storage.Client()
bucket = client.get_bucket('meetup-script')


In [25]:
for blob in list(client.list_blobs(bucket)):
  print(blob.name)

meetup_cookies_chrome_2020-09-19.pkl


In [17]:
import gcsfs
fs = gcsfs.GCSFileSystem(project='	robotic-tiger-270605')
fs.ls('meetup-script')

_call non-retriable exception: Anonymous caller does not have storage.objects.list access to the Google Cloud Storage bucket.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/gcsfs/core.py", line 507, in _call
    self.validate_response(status, contents, json, path, headers)
  File "/usr/local/lib/python3.6/dist-packages/gcsfs/core.py", line 1228, in validate_response
    raise HttpError(error)
gcsfs.utils.HttpError: Anonymous caller does not have storage.objects.list access to the Google Cloud Storage bucket.


HttpError: ignored

In [6]:
import gcsfs
fs = gcsfs.GCSFileSystem(project='my-google-project')
fs.ls('my-bucket')
with fs.open('my-bucket/my-file.txt', 'rb') as f:
  print(f.read())
pd.read_csv("gs://meetup-script/meetup_cookies_chrome_2020-09-19.pkl")

_call non-retriable exception: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/gcsfs/core.py", line 507, in _call
    self.validate_response(status, contents, json, path, headers)
  File "/usr/local/lib/python3.6/dist-packages/gcsfs/core.py", line 1228, in validate_response
    raise HttpError(error)
gcsfs.utils.HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object.


HttpError: ignored

In [2]:
import csv
from io import StringIO

from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.get_bucket('meetup-script')

blob = bucket.blob('meetup_cookies_chrome_2020-09-19.pkl')
blob = blob.download_as_string()
blob = blob.decode('utf-8')

blob = StringIO(blob)  #tranform bytes to string here


OSError: ignored

In [ ]:
# can run these in Dockerfile
# selenium and chromium install
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 11 not upgraded.
Need to get 79.2 MB of archives.
After this operation, 268 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 85.0.4183.83-0ubuntu0.18.04.2 [1,118 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 85.0.4183.83-0ubuntu0.18.04.2 [70.3 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser-l10n all 85.0.418

In [5]:
# authenticate for colab to look at gcs
from google.colab import auth
auth.authenticate_user()
print("auth'ed for colab")

auth'ed for colab


In [ ]:
from time import sleep
import pickle
import selenium.webdriver
# use tf to read files from gcs
from tensorflow.python.lib.io import file_io

# the group to look inside
#TEST_URL   = 'https://www.meetup.com/en-AU/Fishburners-Meetup/events/'
TOP_SHOT_EVENTS_URL = 'https://www.meetup.com/Topshot-Badminton-Meetup/events/'

# cookie location local
#cookie_name = 'meetup_cookies_chrome_2020-09-19.pkl'

# cookie location on gcs
cookie_gcs = 'gs://meetup-script/meetup_cookies_chrome_2020-09-19.pkl'

# picking the chrome driver
OPTIONS = webdriver.ChromeOptions()
# remove all these when watching it the first time
OPTIONS.add_argument('-headless') # no browser
OPTIONS.add_argument('-no-sandbox') # not a test browser
OPTIONS.add_argument('-disable-dev-shm-usage') # so docker will allocate >64MB to chrome
# if we need to be a realer browser (not a headless chrome)
OPTIONS.add_argument('--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""')
driver = webdriver.Chrome('chromedriver',options=OPTIONS)
print('chromedriver - loaded')

# load the page without the cookie
driver.get('https://meetup.com')
print('meetup.com - loaded without cookie')
# load the cookie from google cloud storage
with file_io.FileIO(cookie_gcs,'rb') as f:
  cookies = pickle.load(f)    
for cookie in cookies:
    driver.add_cookie(cookie)
# load the page again after the cookie is loaded, this is the same as logging in
driver.get('https://meetup.com')
print('meetup.com - loaded with cookie')

# go to our meetup group
driver.get(TOP_SHOT_EVENTS_URL)
print(f'meetup group {TOP_SHOT_EVENTS_URL} - loaded')

# find events to attend, this will be in a funky format
elem = driver.find_elements_by_partial_link_text("Attend")
# grab the actual URLs and remove the ?action bit
elems = []
for e in elem[:4]:
  # remove the action rsvp yes part, because that doesn't seem to work (19 Sep 2020)
  elems.append(e.get_attribute('href').split('?action')[0])

# try attend first 4 events in the list
# hence for topshot having 3x sessions a week
# 4th sessions will always be the new open one
for e in elems[:4]:
  # looks like the action bit only comes up if we're logged in
  if 'action' not in e.split('/')[6]:
    return "ERROR: not logged in"
  # go to the first event page
  driver.get(e)
  print(f'meetup group event - {e} loaded')

  try: 
    # click attend button, but this won't always exist
    driver.find_element_by_class_name('gtmEventFooter--attend-btn').click()
    print('meetup group event - clicked attend')

  except Exception as e:
    print(f'no attend button to click with error: {e}')

  # wait 1/2 a second before trying another
  sleep(0.5)
  print('-----wait 1/2 a second then try next')

chromedriver - loaded
meetup.com - loaded without cookie
meetup.com - loaded with cookie
meetup group URL - loaded
meetup group event - URL loaded
meetup group event - clicked attend
